#Objective
- Identify key factors that contribute to employee turnover, perform predictive modeling to assess employees who are most likely to leave, and provide insights for developing targeted retention strategies based on demographics, performance metrics, job satisfaction levels, and other relevant employee data. This type of project would ultimately help HR departments to proactively address potential attrition risks and improve employee retention rates.


#Method of Analysis
- **Classification**
  -  Analyze the various employee factors and variables to predict whether an employee is likely to leave the company, essentially classifying them as "attritted" or "not attritted" based on the input data.
  - Dependent/Target Feature: `Attrition`
  - Independent/Explanatory Features: All the features in the dataset EXCEPT for `Employee ID` and `Attrition`
  - ML Techniques: **Logistic Regression, Support Vector Machines, K-Nearest Neighbors, along with applying GridSearch**

- **Clustering** (*Possible Addition*)
    - By applying clustering algorithms to employee attrition data, we can identify distinct groups of employees with similar characteristics that are more likely to leave.
    - ML Technique: **K-Means Clustering**


#Dataset Information
###Column Descriptions:
1.	`Employee_ID`: Unique identifier for each employee.
2.	`Age`: Age of the employee.
3.	`Gender`: Gender of the employee.
4.	`Marital_Status`: Marital status of the employee (Single, Married, Divorced).
5.	`Department`: Department the employee works in (e.g., HR, IT, Sales, Marketing).
6.	`Job_Role`: Specific role within the department (e.g., Manager, Analyst).
7.	`Job_Level`: Level in the organizational hierarchy.
8.	`Monthly_Income`: Monthly salary of the employee.
9.	`Hourly_Rate`: Rate per hour for hourly employees.
10.	`Years_at_Company`: Number of years the employee has been with the company.
11.	`Years_in_Current_Role`: Number of years the employee has been in their current role.
12.	`Years_Since_Last_Promotion`: Time since the employee’s last promotion.
13.	`Work_Life_Balance`: Rating of work-life balance.
14.	`Job_Satisfaction`: Rating of job satisfaction (1-5 scale).
15.	`Performance_Rating`: Performance rating (1-5 scale).
16.	`Training_Hours_Last_Year`: Number of training hours completed in the past year.
17.	`Overtime`: Whether the employee works overtime (Yes/No).
18.	`Project_Count`: Number of projects managed by the employee.
19.	`Average_Hours_Worked_Per_Week`: Average working hours per week.
20.	`Absenteeism`: Number of days the employee was absent in the past year.
21.	`Work_Environment_Satisfaction`: Rating of work environment satisfaction.
22.	`Relationship_with_Manager`: Rating of the relationship with the manager.
23.	`Job_Involvement`: Rating of job involvement.
24.	`Distance_From_Home`: Distance from home to the workplace (in kilometers).
25.	`Number_of_Companies_Worked`: No. of companies the employee has worked for.
26.	`Attrition`: (Yes/No), the target variable indicating whether the employee left the company.

###Independent Variables:
- All the features in the dataset **EXCEPT** for `Employee ID` and `Attrition`

###Dependent Variable:
- `Attrition`: (Yes/No), the target feature indicating whether the employee left the company.

###Dataset Source:
- https://www.kaggle.com/datasets/ziya07/employee-attrition-prediction-dataset



---


---

#PySpark Setup for Google Colab

In [ ]:
# Connect drive to Colab
from google.colab import drive
drive.mount('/content/drive')

# Instal JVM
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Install latest Spark version with Hadoop
!wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz

In [ ]:
# Unzip Spark folder
!tar xf spark-3.5.5-bin-hadoop3.tgz

In [ ]:
# Install findspark to enable Spark imported as a regular library
!pip install -q findspark

In [ ]:
# Set environmental paths to enable PySpark in the Colab environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [ ]:
# Locate Spark
import findspark
findspark.init()
findspark.find()

In [ ]:
# Create Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

#Data Preprocessing

###Load dataset

In [ ]:
# Import required libraries
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#set plotting backend to matplotlib instead of plotly
ps.set_option('plotting.backend', 'matplotlib')
%matplotlib inline


In [ ]:
# Read data into pyspark dataframe
df = ps.read_csv('/content/employee_attrition_dataset.csv')

In [ ]:
# Show first 20 rows
df.head(20)

In [ ]:
# Display dataset information
df.info()

- Dataset information shows numerical features are the majority.

##Inspecting and Handling Duplicates

###Exact duplicates

In [ ]:
# Inspect to see if we have any rows that are duplicated
print(df.shape[0], df.to_spark().distinct().count())

- Results show 10,000 total rows and 10,000 are all distinct
- This indicates that there are NO duplicates

###Duplicates where only ID differs

In [ ]:
# Remove ID column, then count total rows and distinct to inspect possible duplicates
no_ids = (
    df[[col for col in df.columns if col != 'Employee_ID']]
)

no_ids.shape[0], no_ids.to_spark().distinct().count()

- Results show 10,000 total rows and 10,000 are all distinct
- Again, this indicates that there are NO duplicates

###Inspect Duplicated ID's

In [ ]:
# Inspect if there are duplicaed IDs
df.shape[0], df['Employee_ID'].nunique()

- Results show 10,000 total rows and 10,000 are all distinct
- Again, this indicates that there are NO duplicates

##Inspecting and Handling Missing Values

###Missing values per row

In [ ]:
# Examine if there are any rows with missing values (across rows)
# axis=1, which are columns, because you want to collapse the columns
x = df.isna().sum(axis=1)
x[x >= 1].shape[0]

- Result shows 0 missing values across rows
- We can proceed to check missing values across columns

###Missing values per column

In [ ]:
# Count percentage of missing values in each column
# axis=0, which is rows, because you want to collapse the rows
# this gives percentage of null values in each column

df.isna().sum(axis=0)/df.shape[0]

- Results show all columns have 0% or no null values for each
- Since there are no missing values across both row and columns, there's no need for imputation.

##General Descriptive Statistics

In [ ]:
# Describe numeric features
df.describe()

In [ ]:
# Transposed view
df.describe().transpose()

####Mean and Standard Deviation:
- The mean values vary significantly across features, with some averaging between 2-5, while others go above 11,000. This suggests that different features measure data on very different scales.
- **This is due to the fact that many features have values that are based on a rating scale.**
- The standard deviation also varies widely. Some features have values that are closely around the mean, while others show large variations, indicating more dispersed data.

####Min, Max and Percentiles:
- The minimum values range from 0 to 3,000, while the maximum values reach up to 19,999, emphasizing the presence of different scales across features.
- Looking at the percentiles, the 25% is quite low 1-3 for many features because of the rating scale they're based on. For some, the 75% goes into the thousands, because some of the features are based on larger scales, such as monthly income.

####Distribution Observations:
- Several features appear to be right-skewed, meaning that while most values are small, there are some much larger values pulling the average towards the right or up.
- A few features, however, display a more balanced distribution, where the quartiles are more evenly spaced, meaning data points are more consistently spread.

###Computing Correlations

In [ ]:
# Applying .corr() function to calculate correlation values
df.corr()

####Interpretaion of Correlation Values:
- General Observations:
  - Most correlation values are quite low, indicating weak or no strong linear relationships between the variables.
  - No highly positive (close to 1) or highly negative (close to -1) correlations are present, meaning the dataset does not have obvious strong dependencies among features.
- Key Insights & Relationships:
  - `Work-Life Balance & Job Satisfaction` (0.008876): Although weak, there is a slight positive correlation, suggesting that employees with better work-life balance might experience marginally higher job satisfaction.
  - `Job Satisfaction & Performance Rating` (-0.010438): This weak negative correlation suggests that job satisfaction does not directly impact performance ratings in a significant way.
  - `Years Since Last Promotion & Work-Life Balance` (-0.007506): A slight negative correlation, which may indicate that employees who have not been promoted recently might feel a minor decrease in work-life balance.
  - `Distance from Home & Number of Companies Worked` (-0.018451): A weak negative correlation, which could mean employees who have worked for more companies might tend to live slightly closer to their workplace.
  - `Average Hours Worked Per Week & Absenteeism` (0.012884): A very weak positive correlation suggests that those working more hours per week may have slightly higher absenteeism, but the effect is negligible.

- An alternative approach for this could also be **performing scaling operations first** on the data before computing correlation values; it may be more effective that way.

##Data Visualization

###Histogram

- **Module lecture code does not work on Google Colab environment, so had to develop alternative code.**

In [ ]:
# Define a function to plot histograms of numeric features
def plot_hist(df, features, n_rows, n_cols):
    # Convert the Pandas-on-Spark DataFrame to a Pandas DataFrame
    df_pandas = df[features].to_pandas()

    fig, ax = plt.subplots(n_rows, n_cols, figsize=(16, 12))
    fig.tight_layout(pad=3.0)

    # Flatten ax to handle different subplot configurations
    ax = ax.flatten() if isinstance(ax, np.ndarray) else [ax]

    for i in range(len(features)):
        ax[i].hist(df_pandas[features[i]])  # Now using standard Pandas data
        ax[i].set_title(features[i])

    # Show the plots
    plt.show()

In [ ]:
# Remove Employee ID column
no_id_df = df.drop('Employee_ID', axis=1)

no_id_df.info()

In [ ]:
# Isolate numerical features
num_features = no_id_df.select_dtypes(include=np.number).columns.tolist()

# Apply custom function for histograms
# Plot first half
plot_hist(df=no_id_df, features=num_features[:9], n_rows=5, n_cols=2)

###Interpretations:

-  `Age`: The distribution appears relatively uniform, suggesting that employees' ages are evenly spread across the range.
- `Job Level`: The distribution is discrete with five distinct levels, each having approximately equal frequency, indicating an even spread.
- `Monthly Income`: The income distribution is nearly uniform, meaning salaries are evenly distributed without significant skewness.
- `Hourly Rate`: The distribution is also roughly uniform, suggesting that employees receive a broad but evenly spread range of hourly wages.
- `Years at Company`: The distribution is mostly uniform, except for a slight dip in the middle, indicating a lower number of employees with mid-range tenure.
- `Years in Current Role`: The histogram shows gaps at certain intervals, implying certain clusters where employees tend to remain in their roles for specific durations.
- `Years Since Last Promotion`: The distribution is uniform, showing that promotions are spread evenly across employees over different years.
- `Work-Life Balance`: The data is discrete, with four distinct levels, each having roughly equal frequency, indicating a balanced distribution of work-life satisfaction ratings.
- `Job Satisfaction`: The ratings are evenly distributed across five levels, suggesting no strong bias toward dissatisfaction or satisfaction.

In [ ]:
# Plot the second half

plot_hist(df=no_id_df, features=num_features[9:], n_rows=5, n_cols=2)

###Interpretations:
- `Performance Rating`: The ratings are evenly distributed across four categories, indicating no significant skewness in performance evaluations.
- `Training Hours Last Year`: The distribution appears uniform, suggesting employees received a fairly even spread of training hours.
- `Project Count`: The histogram is nearly uniform, meaning employees have similar range of project work without a clear concentration in any category.
- `Average Hours Worked Per Week`: The distribution is relatively even, implying employees work a balanced range of hours without extreme variations.
- `Absenteeism`: The histogram appears uniform, indicating employees' absenteeism is evenly spread over different values.
- `Work Environment Satisfaction`: The ratings are evenly distributed among four levels, showing no particular bias toward high or low satisfaction.
- `Relationship with Manager`: The distribution is discrete with equal frequencies across four levels, indicating balanced ratings for manager relationships.
- `Job Involvement`: The histogram shows an even distribution across four levels, suggesting no dominant trend or bias in job involvement ratings.
- `Distance from Home`: The values are quite uniformly spread, meaning employees live at varying distances and is not skewed in either end.
- `Number of Companies Worked`: The histogram shows equal distribution across categories, suggesting employees have varied past job experiences without a significant trend.

In [ ]:
df.info()

###Bar Charts for Categorical Columns

In [ ]:
# Convert the Spark DataFrame to Pandas first
bar_df = df.to_pandas()

In [ ]:
# Plot Gender Distribution

plt.figure(figsize=(10,6))
bar_df['Gender'].value_counts().plot.bar()
plt.title('Gender Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

###Interpretation:
- `Gender`: Both categories are quite evenly numbered and bars are about the same height, showing that there's a pretty even amount of both male and female, with female just having slightly more.

In [ ]:
# Plot Marital Status Distribution

plt.figure(figsize=(10,6))
bar_df['Marital_Status'].value_counts().plot.bar(color='orange')
plt.title('Marital Status Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

###Interpretation:
- `Marital Status`: All three cateogries are quite evenly numbered and bars are about the same height, showing that all three are evenly spread across the dataset.

In [ ]:
# Plot Department Distribution

plt.figure(figsize=(10,6))
bar_df['Department'].value_counts().plot.bar(color='red')
plt.title('Department Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

###Interpretation:
- `Department`: All categories have a good even spread, with Marketing having slighlty dominant numbers, and  the rest being evenly distributed.

In [ ]:
# Plot Job Roles Distribution

plt.figure(figsize=(10,6))
bar_df['Job_Role'].value_counts().plot.bar(color='green')
plt.title('Job Roles Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

###Interpretation:
- `Job Roles`: All categories are roughly even, with Analyst and Assistant roles having just slighlty dominant numbers, and decreasing trend towards Managerial roles, but the differences are not quite significant.

In [ ]:
# Plot Overtime Distribution

plt.figure(figsize=(10,6))
bar_df['Overtime'].value_counts().plot.bar(color='brown')
plt.title('Overtime Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

###Interpretation:
- `Overtime`: People who did not take overtime have a slight dominance but the difference is not really that significant.

In [ ]:
# Plot Attrition Distribution

plt.figure(figsize=(10,6))
bar_df['Attrition'].value_counts().plot.bar(color='purple')
plt.title('Attrition Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

###Interpretation:
- `Attrition`: People who left the company appear to be a significantly dominant trend in the dataset; the difference is in the thousands.

###Scatterplots

- **Module lecture code does not work on Google Colab environment, so had to develop alternative code.**

In [ ]:
# Convert the Spark DataFrame to Pandas first
scatter_df = df.to_pandas()

# Use Pandas scatter plot
# Plot Years at Company vs Monthly Income
plt.figure(figsize=(10,6))

plt.scatter(data=scatter_df, x='Years_at_Company', y='Monthly_Income')
plt.title('Years at Company vs Monthly Income')
plt.show()

###Interpretation:
- `Years at Company vs Monthly Income`: Scatterplot shows no relationship at all, with data points being spread out all over the chart.

In [ ]:
# Plot Work Life Balance and Job Satisfaction

plt.figure(figsize=(10,6))
plt.scatter(data=scatter_df, x='Work_Life_Balance', y='Job_Satisfaction')
plt.title('WLB vs Job Satisfaction')
plt.show()

###Interpretation:
- `WLB vs Job Satisfaction`: Shows no relationship at all, with data points scattered around the chart, indicating that there's no linearity.

In [ ]:
# Plot Job Satisfaction and Performance Rating

plt.figure(figsize=(10,6))

plt.scatter(data=scatter_df, x='Job_Satisfaction', y='Performance_Rating' )
plt.title('Job Satisfaction vs Performance Rating')
plt.show()

###Interpretation:
- `Job Satisfaction vs Performance Rating`: Shows no relationship at all, with data points scattered all around the chart, indicating inexistence of linearity.

In [ ]:
# Plot Work Environment Satisfaction and Performance Rating

plt.figure(figsize=(10,6))
plt.scatter(data=scatter_df, x='Work_Environment_Satisfaction', y='Performance_Rating')
plt.title('Work Environment Satisfaction vs Performance Rating')
plt.show()

###Interpretation:
- `Work Environment Satisfaction vs Performance Rating `: Shows no relationship at all, with data points being spread all over the chart, displaying no linearity.

In [ ]:
# Plot Avg Hours Worked per Week and Absenteeism

plt.figure(figsize=(10,6))
plt.scatter(data=scatter_df, x='Average_Hours_Worked_Per_Week', y='Absenteeism')
plt.title('Avg Hours Worked Per Week vs Absenteeism')
plt.show()

###Interpretation:
- `Avg Hours Worked Per Week vs Absenteeism`: Plot shows no relationship at all, with data points being spread out all over the chart axes, which again shows no linearity.

##Handling Outliers
- We'll utilize **Box Plots** and **refer to the Histograms above** to assess outliers.

In [ ]:
# Convert the Spark DataFrame to Pandas first
box_plot_df = df.to_pandas()

In [ ]:
# Use box plot

plt.figure(figsize=(8,6))
plt.boxplot(data=box_plot_df, x='Monthly_Income' )
plt.title('Monthly Income Box Plot')
plt.show()

###Interpretation:
- `Monthly Income Box Plot`: Shows no outliers in either end.
- For this analysis, there's no need for handling outliers based on Monthly Income because there are none.

In [ ]:
plt.figure(figsize=(8,6))
plt.boxplot(data=box_plot_df, x='Hourly_Rate' )
plt.title('Hourly Rate Box Plot')
plt.show()

###Interpretation:
- `Hourly Rate Box Plot`: Shows no outliers in either end.
- For this analysis, there's no need for handling outliers based on Hourly Rate because there are none.

In [ ]:
plt.figure(figsize=(10,6))
plt.boxplot(box_plot_df[['Years_in_Current_Role', 'Years_at_Company','Years_Since_Last_Promotion']])
plt.title('Years in Current Role(1), Years at Company(2), Years Since Last Promotion(3) Box Plots')
plt.show()

###Interpretation:
- `Years in Current Role(1), Years at Company(2), Years Since Last Promotion(3) Box Plots`: Shows no outliers on all three features.
- For this analysis, there's no need for handling outliers based on these features because there are none.

In [ ]:
plt.figure(figsize=(10,6))
plt.boxplot(box_plot_df[['Average_Hours_Worked_Per_Week','Training_Hours_Last_Year','Absenteeism']] )
plt.title('Avg Hours Worked Per Week(1), Training Hours Last Year(2), Absenteeism(3) Box Plots')

###Interpretation:
- `Avg Hours Worked Per Week(1), Training Hours Last Year(2), Absenteeism(3) Box Plots`: Shows no outliers on all three features.
- For this analysis, there's no need for handling outliers based on these features because there are none.

###Outlier Analysis Decision:
- Based on the box plots above, there are **NO OUTLIERS** in all the seemingly important features in the dataset, considering the topic of the project.
- The histograms plotted above also displays **NO SIGNIFICANT SKEWNESS** in all the features plotted, they are all roughly uniform and evenly distributed in the features with distinct levels or categories, and no clear dominance in frequencies.
- Considering the insights gained from this analysis, there's no need for handling outliers because there is none, and we previously determined that there are no missing values, so **all of the data will be retained and utilized**.